# FF P2PY MPI

In [1]:
%%writefile nc2cp.f90
subroutine fs(ss, ll, ts, tf, tt, mr, ms)
    use, intrinsic :: iso_c_binding
    use MPI
    implicit none
    include 'fftw3-mpi.f03'
    integer, intent(out) :: mr, ms, ll
    double complex, intent(out) :: ss
    double precision, intent(out) :: ts, tf, tt
    integer :: mpirank, mpisize, mpierror, i, j, k
    integer(C_INTPTR_T), parameter :: L = 576, M = 576, N = 576
    type(C_PTR) :: plan, cdata
    complex(C_DOUBLE_COMPLEX), pointer :: data(:,:,:)
    integer(C_INTPTR_T) :: alloc_local, local_N, local_start
    complex(C_DOUBLE_COMPLEX) :: s, rs
    real(C_DOUBLE) :: t0, t1, t2

    call cpu_time(t0)    ! time measurement
    
    call MPI_Init(mpierror)
    call MPI_Comm_rank(MPI_COMM_WORLD, mpirank, mpierror)
    call MPI_Comm_size(MPI_COMM_WORLD, mpisize, mpierror)

    ! init
    call fftw_mpi_init()    

    ! get local data size and allocate (note dimension reversal)
    alloc_local = fftw_mpi_local_size_3d(N, M, L,  &
                 MPI_COMM_WORLD, local_N, local_start)
    cdata = fftw_alloc_complex(alloc_local)
    call c_f_pointer(cdata, data, [L, M, local_N])

    ! create MPI plan for in-place forward DFT (note dimension reversal)
    plan = fftw_mpi_plan_dft_3d(N, M, L, data, data,  &
                MPI_COMM_WORLD, FFTW_FORWARD, FFTW_ESTIMATE)

    ! fill array with complex values
    do k = 1, int(local_N)
        do j = 1, M
            do i = 1, L
                data(i, j, k) = sin( real(i + j + (k + local_start)) )
            enddo
        enddo
    enddo
    data = dcmplx( real(data), 0 )
    
    call cpu_time(t1)    ! time measurement

    ! compute transform (as many times as desired)
    call fftw_mpi_execute_dft(plan, data, data)

    ! compute the checksum of processes
    s = sum(data)
    call MPI_Reduce(s,                   &! send data
                    rs,                  &! recv data
                    1,                   &! count
                    MPI_DOUBLE_COMPLEX,  &! data type
                    MPI_SUM,             &! operation
                    0,                   &! rank of root process
                    MPI_COMM_WORLD, mpierror)
    
    ! clean
    call fftw_destroy_plan(plan)
    call fftw_free(cdata)
    call fftw_mpi_cleanup()
    call mpi_finalize(mpierror)
       
    call cpu_time(t2)    ! time measurement        
    
    ! result
    ss = rs * 1e-5
    ll = L
    ts = t1 - t0
    tf = t2 - t1   
    tt = t2 - t0
    mr = mpirank
    ms = mpisize
    
end subroutine

Overwriting nc2cp.f90


In [2]:
%%bash
module load  openmpi/gnu/4.0.4_ucx_1.6  mathlibs/fftw/3.3.8_openmpi-3.1_gnu
export FFTW_ROOT=/scratch/app/mathlibs/fftw/3.3.8_openmpi-3.1_gnu
f2py  -c nc2cp.f90  -m nc2cp  --f90exec=mpif90  --opt='-O3'  --quiet  \
      -L$FFTW_ROOT/lib  -lfftw3_mpi  -lfftw3  -lm  -I$FFTW_ROOT/include  \
      -DNPY_NO_DEPRECATED_API=NPY_1_7_API_VERSION

{'before': '', 'this': 'use', 'after': ', intrinsic :: iso_c_binding '}


In [3]:
%%writefile nc2cp_c.py
import nc2cp
import time as tm
t2 = tm.time()    # time measurement
ss, ll, ts, tf, tt, mr, ms = nc2cp.fs()
t3 = tm.time()    # time measurement
if mr == 0 :
    print(f"S:{ss:.0f}", end='')
    print(f", L:{ll:0g}", end='')
    print(f", N:{ms:0g}", end='')
    print(f", T1:{ts:.4f}", end='')
    print(f", TF:{tf:.4f}", end='')
    print(f", TT:{tt:.4f}", end='')
    print(f", TO:{t3-t2:.4f}")

Overwriting nc2cp_c.py


In [6]:
! cp nc2cp* /scratch${PWD#"/prj"}

In [7]:
%%writefile nc2cp.srm
#!/bin/bash
#SBATCH --job-name nc2cp       # Job name
#SBATCH --partition cpu_small  # Select partition
#SBATCH --ntasks=1             # Total tasks
#SBATCH --time=00:06:00        # Limit execution time
#SBATCH --exclusive            # Exclusive acccess to nodes

echo '========================================'
echo '- Job ID:' $SLURM_JOB_ID
echo '- Tasks per node:' $SLURM_NTASKS_PER_NODE
echo '- # of nodes in the job:' $SLURM_JOB_NUM_NODES
echo '- # of tasks:' $SLURM_NTASKS
echo '- Dir from which sbatch was invoked:' ${SLURM_SUBMIT_DIR##*/}
cd $SLURM_SUBMIT_DIR
echo -n '- List of nodes allocated to the job: '
nodeset -e $SLURM_JOB_NODELIST

# Environment
echo '-- modules ----------------------------'
cd /scratch${PWD#"/prj"}/
module load mathlibs/fftw/3.3.8_openmpi-3.1_gnu
                                              
# Executable
EXEC="python nc2cp_c.py"

# Start
echo '-- run --------------------------------'
echo '$ srun --mpi=pmi2 -n' $SLURM_NTASKS ${EXEC##*/}
echo '-- output -----------------------------'
srun --mpi=pmi2 -n $SLURM_NTASKS $EXEC
echo '~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~'

Overwriting nc2cp.srm


<hr style="height:10px;border-width:0;background-color:green">

### 1 of (1, 4, 16, 24, 48, 72, 96)

In [13]:
! sbatch --ntasks=1 nc2cp.srm
! sbatch --ntasks=1 nc2cp.srm
! sbatch --ntasks=1 nc2cp.srm

Submitted batch job 1331069
Submitted batch job 1331070
Submitted batch job 1331071


In [8]:
! cat /scratch${PWD#"/prj"}/slurm-1331069.out
! cat /scratch${PWD#"/prj"}/slurm-1331070.out
! cat /scratch${PWD#"/prj"}/slurm-1331071.out

- Job ID: 1331069
- Tasks per node:
- # of nodes in the job: 1
- # of tasks: 1
- Dir from which sbatch was invoked: fft
- List of nodes allocated to the job: sdumont1478
-- modules ----------------------------
-- run --------------------------------
$ srun --mpi=pmi2 -n 1 python nc2cp_c.py
-- output -----------------------------
S:270+0j, L:576, N:1, T1:7.7512, TF:16.1642, TT:23.9155, TO:30.1821
~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
- Job ID: 1331070
- Tasks per node:
- # of nodes in the job: 1
- # of tasks: 1
- Dir from which sbatch was invoked: fft
- List of nodes allocated to the job: sdumont1478
-- modules ----------------------------
-- run --------------------------------
$ srun --mpi=pmi2 -n 1 python nc2cp_c.py
-- output -----------------------------
S:270+0j, L:576, N:1, T1:7.7365, TF:16.1256, TT:23.8621, TO:24.2596
~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
- Job ID: 1331071
- Tasks per node:
- # of nodes in the job: 1
- # of tasks: 1
- Dir from which sbatch was invoked: fft
- 

<hr style="height:10px;border-width:0;background-color:green">

### 4 of (1, 4, 16, 24, 48, 72, 96)

In [14]:
! sbatch --ntasks=4 nc2cp.srm
! sbatch --ntasks=4 nc2cp.srm
! sbatch --ntasks=4 nc2cp.srm

Submitted batch job 1331073
Submitted batch job 1331074
Submitted batch job 1331075


In [7]:
! cat /scratch${PWD#"/prj"}/slurm-1331073.out
! cat /scratch${PWD#"/prj"}/slurm-1331074.out
! cat /scratch${PWD#"/prj"}/slurm-1331075.out

- Job ID: 1331073
- Tasks per node:
- # of nodes in the job: 1
- # of tasks: 4
- Dir from which sbatch was invoked: fft
- List of nodes allocated to the job: sdumont1478
-- modules ----------------------------
-- run --------------------------------
$ srun --mpi=pmi2 -n 4 python nc2cp_c.py
-- output -----------------------------
S:270-0j, L:576, N:4, T1:2.1918, TF:4.4423, TT:6.6341, TO:8.0733
~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
- Job ID: 1331074
- Tasks per node:
- # of nodes in the job: 1
- # of tasks: 4
- Dir from which sbatch was invoked: fft
- List of nodes allocated to the job: sdumont1478
-- modules ----------------------------
-- run --------------------------------
$ srun --mpi=pmi2 -n 4 python nc2cp_c.py
-- output -----------------------------
S:270-0j, L:576, N:4, T1:2.1849, TF:4.4276, TT:6.6126, TO:6.8889
~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
- Job ID: 1331075
- Tasks per node:
- # of nodes in the job: 1
- # of tasks: 4
- Dir from which sbatch was invoked: fft
- List o

<hr style="height:10px;border-width:0;background-color:green">

### 16 of (1, 4, 16, 24, 48, 72, 96)

In [15]:
! sbatch --ntasks=16 nc2cp.srm
! sbatch --ntasks=16 nc2cp.srm
! sbatch --ntasks=16 nc2cp.srm

Submitted batch job 1331076
Submitted batch job 1331077
Submitted batch job 1331078


In [6]:
! cat /scratch${PWD#"/prj"}/slurm-1331076.out
! cat /scratch${PWD#"/prj"}/slurm-1331077.out
! cat /scratch${PWD#"/prj"}/slurm-1331078.out

- Job ID: 1331076
- Tasks per node:
- # of nodes in the job: 1
- # of tasks: 16
- Dir from which sbatch was invoked: fft
- List of nodes allocated to the job: sdumont1498
-- modules ----------------------------
-- run --------------------------------
$ srun --mpi=pmi2 -n 16 python nc2cp_c.py
-- output -----------------------------
S:270-0j, L:576, N:16, T1:1.2169, TF:1.5892, TT:2.8060, TO:3.5611
~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
- Job ID: 1331077
- Tasks per node:
- # of nodes in the job: 1
- # of tasks: 16
- Dir from which sbatch was invoked: fft
- List of nodes allocated to the job: sdumont1498
-- modules ----------------------------
-- run --------------------------------
$ srun --mpi=pmi2 -n 16 python nc2cp_c.py
-- output -----------------------------
S:270-0j, L:576, N:16, T1:1.2295, TF:1.5784, TT:2.8080, TO:3.7301
~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
- Job ID: 1331078
- Tasks per node:
- # of nodes in the job: 1
- # of tasks: 16
- Dir from which sbatch was invoked: fft
-

<hr style="height:10px;border-width:0;background-color:green">

### 24 of (1, 4, 16, 24, 48, 72, 96)

In [16]:
! sbatch --ntasks=24 nc2cp.srm
! sbatch --ntasks=24 nc2cp.srm
! sbatch --ntasks=24 nc2cp.srm

Submitted batch job 1331079
Submitted batch job 1331080
Submitted batch job 1331081


In [5]:
! cat /scratch${PWD#"/prj"}/slurm-1331079.out
! cat /scratch${PWD#"/prj"}/slurm-1331080.out
! cat /scratch${PWD#"/prj"}/slurm-1331081.out

- Job ID: 1331079
- Tasks per node:
- # of nodes in the job: 1
- # of tasks: 24
- Dir from which sbatch was invoked: fft
- List of nodes allocated to the job: sdumont1450
-- modules ----------------------------
-- run --------------------------------
$ srun --mpi=pmi2 -n 24 python nc2cp_c.py
-- output -----------------------------
S:270+0j, L:576, N:24, T1:1.3558, TF:1.1128, TT:2.4686, TO:4.2907
~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
- Job ID: 1331080
- Tasks per node:
- # of nodes in the job: 1
- # of tasks: 24
- Dir from which sbatch was invoked: fft
- List of nodes allocated to the job: sdumont1498
-- modules ----------------------------
-- run --------------------------------
$ srun --mpi=pmi2 -n 24 python nc2cp_c.py
-- output -----------------------------
S:270+0j, L:576, N:24, T1:1.3925, TF:1.1246, TT:2.5172, TO:4.0417
~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
- Job ID: 1331081
- Tasks per node:
- # of nodes in the job: 1
- # of tasks: 24
- Dir from which sbatch was invoked: fft
-

<hr style="height:10px;border-width:0;background-color:green">

### 48 of (1, 4, 16, 24, 48, 72, 96)

In [17]:
! sbatch --ntasks=48 nc2cp.srm
! sbatch --ntasks=48 nc2cp.srm
! sbatch --ntasks=48 nc2cp.srm

Submitted batch job 1331082
Submitted batch job 1331083
Submitted batch job 1331084


In [4]:
! cat /scratch${PWD#"/prj"}/slurm-1331082.out
! cat /scratch${PWD#"/prj"}/slurm-1331083.out
! cat /scratch${PWD#"/prj"}/slurm-1331084.out

- Job ID: 1331082
- Tasks per node:
- # of nodes in the job: 2
- # of tasks: 48
- Dir from which sbatch was invoked: fft
- List of nodes allocated to the job: sdumont1480 sdumont1481
-- modules ----------------------------
-- run --------------------------------
$ srun --mpi=pmi2 -n 48 python nc2cp_c.py
-- output -----------------------------
S:270+0j, L:576, N:48, T1:1.4770, TF:0.9400, TT:2.4170, TO:4.4530
~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
- Job ID: 1331083
- Tasks per node:
- # of nodes in the job: 2
- # of tasks: 48
- Dir from which sbatch was invoked: fft
- List of nodes allocated to the job: sdumont1443 sdumont1444
-- modules ----------------------------
-- run --------------------------------
$ srun --mpi=pmi2 -n 48 python nc2cp_c.py
-- output -----------------------------
S:270+0j, L:576, N:48, T1:1.3645, TF:0.9260, TT:2.2905, TO:4.1233
~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
- Job ID: 1331084
- Tasks per node:
- # of nodes in the job: 2
- # of tasks: 48
- Dir from which s

<hr style="height:10px;border-width:0;background-color:green">

### 72 of (1, 4, 16, 24, 48, 72, 96)

In [18]:
! sbatch --ntasks=72 nc2cp.srm
! sbatch --ntasks=72 nc2cp.srm
! sbatch --ntasks=72 nc2cp.srm

Submitted batch job 1331085
Submitted batch job 1331086
Submitted batch job 1331087


In [3]:
! cat /scratch${PWD#"/prj"}/slurm-1331085.out
! cat /scratch${PWD#"/prj"}/slurm-1331086.out
! cat /scratch${PWD#"/prj"}/slurm-1331087.out

- Job ID: 1331085
- Tasks per node:
- # of nodes in the job: 3
- # of tasks: 72
- Dir from which sbatch was invoked: fft
- List of nodes allocated to the job: sdumont1447 sdumont1448 sdumont1482
-- modules ----------------------------
-- run --------------------------------
$ srun --mpi=pmi2 -n 72 python nc2cp_c.py
-- output -----------------------------
S:270-0j, L:576, N:72, T1:1.3428, TF:0.9832, TT:2.3260, TO:4.5457
~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
- Job ID: 1331086
- Tasks per node:
- # of nodes in the job: 3
- # of tasks: 72
- Dir from which sbatch was invoked: fft
- List of nodes allocated to the job: sdumont1479 sdumont1480 sdumont1481
-- modules ----------------------------
-- run --------------------------------
$ srun --mpi=pmi2 -n 72 python nc2cp_c.py
-- output -----------------------------
S:270-0j, L:576, N:72, T1:1.3156, TF:0.9308, TT:2.2464, TO:2.9738
~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
- Job ID: 1331087
- Tasks per node:
- # of nodes in the job: 3
- # of task

<hr style="height:10px;border-width:0;background-color:green">

### 96 of (1, 4, 16, 24, 48, 72, 96)

In [19]:
! sbatch --ntasks=96 nc2cp.srm
! sbatch --ntasks=96 nc2cp.srm
! sbatch --ntasks=96 nc2cp.srm

Submitted batch job 1331088
Submitted batch job 1331089
Submitted batch job 1331090


In [2]:
! cat /scratch${PWD#"/prj"}/slurm-1331088.out
! cat /scratch${PWD#"/prj"}/slurm-1331089.out
! cat /scratch${PWD#"/prj"}/slurm-1331090.out

- Job ID: 1331088
- Tasks per node:
- # of nodes in the job: 4
- # of tasks: 96
- Dir from which sbatch was invoked: fft
- List of nodes allocated to the job: sdumont1446 sdumont1447 sdumont1448 sdumont1450
-- modules ----------------------------
-- run --------------------------------
$ srun --mpi=pmi2 -n 96 python nc2cp_c.py
-- output -----------------------------
S:270-0j, L:576, N:96, T1:1.3494, TF:1.0494, TT:2.3988, TO:3.3963
~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
- Job ID: 1331089
- Tasks per node:
- # of nodes in the job: 4
- # of tasks: 96
- Dir from which sbatch was invoked: fft
- List of nodes allocated to the job: sdumont1479 sdumont1480 sdumont1481 sdumont1482
-- modules ----------------------------
-- run --------------------------------
$ srun --mpi=pmi2 -n 96 python nc2cp_c.py
-- output -----------------------------
S:270-0j, L:576, N:96, T1:1.2645, TF:1.0391, TT:2.3035, TO:6.0034
~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
- Job ID: 1331090
- Tasks per node:
- # of nodes i

<hr style="height:10px;border-width:0;background-color:red">